### Import Python Packages to start reproducing results with ABCD graphs

In [508]:
### Ignore user warnings
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [509]:
### import packages
import os
import numpy as np
import scipy as sp
import pandas as pd
import networkx as nx
from sklearn.metrics import adjusted_mutual_info_score
### For MR and MV
import pygenstability as pgs
from pygenstability import plotting as pgs_plotting
### For Bayan
import bayanpy as bp
### For Louvain, Leiden, Paris, Combo
from cdlib import algorithms

### Seed the random number generator
np.random.seed(42)


### Define Algorithms calculating AMI score for each ABCD Graphs

In [510]:
### Run PyGenStability's Markov Stability with automated optimal scale selection using NVI
def MV_Run(G):
    ### Convert NetworkX graph into csgraph for PyGenStability to use
    A = nx.to_scipy_sparse_array(G)

    ### Run PyGenStability's Markov Stability with automated optimal scale selection using NVI
    MV_results = pgs.run(
        A,
        n_scale=50,
        constructor='continuous_combinatorial',
        with_optimal_scales=True,
        n_workers=4,
        tqdm_disable=True
    )

    ### Selected the partition with the minimum NVI
    ### Initialize the minimum NVI to 1 since the max NVI is 1
    min_NVI = 1
    min_NVI_partition = MV_results['selected_partitions'][0]
    for i in MV_results['selected_partitions']:
        min_NVI = min(min_NVI, MV_results['NVI'][i])
        if MV_results['NVI'][i] == min_NVI:
            min_NVI_partition = i

    ### Convert the best partitions to numpy array
    partition_pred = np.array(MV_results['community_id'][min_NVI_partition])

    ### Read the community labels from the networkX graph and create the true partition array
    community_dict = {}
    index = 0
    for node in G.nodes:
        if G.nodes[node]['community'] not in community_dict.values():
            community_dict[index] = G.nodes[node]['community']
            index += 1

    partition_true = np.array([key for node in G.nodes for key, value in community_dict.items() if G.nodes[node]['community'] == value])
    ami_score = adjusted_mutual_info_score(partition_true, partition_pred)
            
    return ami_score

In [511]:
### Run PyGenStability's Markov Stability with a random partition
def MR_Run(G):
    ### Convert NetworkX graph into csgraph for PyGenStability to use
    A = nx.to_scipy_sparse_array(G)

    ### Run PyGenStability's Markov Stability with automated optimal scale selection using NVI
    MV_results = pgs.run(
        A,
        n_scale=50,
        constructor='continuous_combinatorial',
        with_optimal_scales=True,
        n_workers=4,
        tqdm_disable=True
    )

    ### Randomly select a partition from 0-49 because the n_scale is 50
    random_partition = np.random.randint(0, 50)

    ### Convert the best partitions to numpy array
    partition_pred = np.array(MV_results['community_id'][random_partition])

    ### Read the community labels from the networkX graph and create the true partition array
    community_dict = {}
    index = 0
    for node in G.nodes:
        if G.nodes[node]['community'] not in community_dict.values():
            community_dict[index] = G.nodes[node]['community']
            index += 1

    partition_true = np.array([key for node in G.nodes for key, value in community_dict.items() if G.nodes[node]['community'] == value])
    ami_score = adjusted_mutual_info_score(partition_true, partition_pred)
            
    return ami_score

In [512]:
### Run Bayan to find partition within 1% of the maximum modularity with a execution time limit of 60 seconds and resolution parameter of 1
def Bayan_Run(G):
    
    modularity, optimality_gap, community, modeling_time, solve_time = bp.bayan(G, threshold=0.01, time_allowed=60, resolution=1)

    ### Read the community labels from the networkX graph and create the true partition array
    community_dict = {}
    index = 0
    for node in G.nodes:
        if G.nodes[node]['community'] not in community_dict.values():
            community_dict[index] = G.nodes[node]['community']
            index += 1

    partition_true = np.array([key for node in G.nodes for key, value in community_dict.items() if G.nodes[node]['community'] == value])

    ### Create the predicted partition array, since the community labels are not continuous, use integer type
    partition_pred = np.zeros(len(G.nodes), dtype=int)
    for i in range(len(community)):
        for node in G.nodes:
            if node in community[i]:
                partition_pred[int(node)] = i

    ### Calculate the AMI score
    ami_score = adjusted_mutual_info_score(partition_true, partition_pred)
    return ami_score



In [513]:
### Run Louvain's algorithm to find partition
def Louvain_Run(G):
    _cdlib_global_seed = 42
    communities = algorithms.louvain(G, randomize=42)

    ### Create the predicted partition array from Louvain
    partition_pred = np.zeros(len(G.nodes), dtype=int)
    for index, community in enumerate(communities.communities):
        for node in community:
            partition_pred[int(node)] = index


    ### Read the community labels from the networkX graph and create the true partition array
    community_dict = {}
    index = 0
    for node in G.nodes:
        if G.nodes[node]['community'] not in community_dict.values():
            community_dict[index] = G.nodes[node]['community']
            index += 1

    partition_true = np.array([key for node in G.nodes for key, value in community_dict.items() if G.nodes[node]['community'] == value])

    ### Calculate the AMI score
    ami_score = adjusted_mutual_info_score(partition_true, partition_pred)
    
    return ami_score


In [514]:
### Run Leiden's algorithm to find partition
### Since in CDLib 0.2.6, the Leiden algorithm does not have a seed parameter, and it is utilizing leidenalg package to 
### perform the Leiden algorithm which is not deterministic and is taking random seed as a parameter, 
### and there is no global seed for leidenalg package,
### therefore, by using CDLib 0.2.6, there is no way to fix the seed for the Leiden algorithm unless we use leidenalg package directly.
def Leiden_Run(G):
    communities = algorithms.leiden(G)

    ### Create the predicted partition array from Leiden
    partition_pred = np.zeros(len(G.nodes), dtype=int)
    for index, community in enumerate(communities.communities):
        for node in community:
            partition_pred[int(node)] = index

    ### Read the community labels from the networkX graph and create the true partition array
    community_dict = {}
    index = 0
    for node in G.nodes:
        if G.nodes[node]['community'] not in community_dict.values():
            community_dict[index] = G.nodes[node]['community']
            index += 1

    partition_true = np.array([key for node in G.nodes for key, value in community_dict.items() if G.nodes[node]['community'] == value])

    ### Calculate the AMI score
    ami_score = adjusted_mutual_info_score(partition_true, partition_pred)
    
    return ami_score


In [515]:
### Run Paris's algorithm to find partition
def Paris_Run(G):
    communities = algorithms.paris(G)

    ### Create the predicted partition array from Paris
    partition_pred = np.zeros(len(G.nodes), dtype=int)
    for index, community in enumerate(communities.communities):
        for node in community:
            partition_pred[int(node)] = index

    ### Read the community labels from the networkX graph and create the true partition array
    community_dict = {}
    index = 0
    for node in G.nodes:
        if G.nodes[node]['community'] not in community_dict.values():
            community_dict[index] = G.nodes[node]['community']
            index += 1

    partition_true = np.array([key for node in G.nodes for key, value in community_dict.items() if G.nodes[node]['community'] == value])

    ### Calculate the AMI score
    ami_score = adjusted_mutual_info_score(partition_true, partition_pred)

    return ami_score


In [516]:
### Run Combo algorithm to find partition
def Combo_Run(G):
    communities = algorithms.pycombo(G)

    ### Create the predicted partition array from Combo
    partition_pred = np.zeros(len(G.nodes), dtype=int)
    for index, community in enumerate(communities.communities):
        for node in community:
            partition_pred[int(node)] = index

    ### Read the community labels from the networkX graph and create the true partition array
    community_dict = {}
    index = 0
    for node in G.nodes:
        if G.nodes[node]['community'] not in community_dict.values():
            community_dict[index] = G.nodes[node]['community']
            index += 1

    partition_true = np.array([key for node in G.nodes for key, value in community_dict.items() if G.nodes[node]['community'] == value])

    ### Calculate the AMI score
    ami_score = adjusted_mutual_info_score(partition_true, partition_pred)

    return ami_score

In [517]:
### Run Infomap to find partition
def Infomap_Run(G):
    communities = algorithms.infomap(G)

    ### Create the predicted partition array from Belief
    partition_pred = np.zeros(len(G.nodes), dtype=int)
    for index, community in enumerate(communities.communities):
        for node in community:
            partition_pred[int(node)] = index

    ### Read the community labels from the networkX graph and create the true partition array
    community_dict = {}
    index = 0
    for node in G.nodes:
        if G.nodes[node]['community'] not in community_dict.values():
            community_dict[index] = G.nodes[node]['community']
            index += 1

    partition_true = np.array([key for node in G.nodes for key, value in community_dict.items() if G.nodes[node]['community'] == value])

    ### Calculate the AMI score
    ami_score = adjusted_mutual_info_score(partition_true, partition_pred)

    return ami_score

### Now we start running test for ABCD graphs with mixing parameter = 0.1, 0.3, 0.5, 0.7, 0.9
### And calculate the average AMI scores for each algorithm under different mixing parameters

In [ ]:
def run_algorithms_on_ABCD(mixing_parameter):
    """
    Run community detection algorithms on ABCD graphs with specified mixing parameter
    
    Args:
        mixing_parameter (float): The mixing parameter value (e.g. 0.1)
        
    Returns:
        dict: Dictionary containing AMI scores for each graph and for each algorithm
    """
    ### Initialize results storage
    results = {
        'MV': [],
        'MR': [], 
        'Bayan': [],
        'Louvain': [],
        'Leiden': [],
        'Paris': [],
        'Combo': [],
        'Infomap': []
    }

    ### Loop through all ABCD graphs from 0 to 99
    for graph_num in range(100):
        
        ### Load the graph
        gml_path = f'./FigShare_repo/Random_graph_data/ABCD_graphs/xi_{mixing_parameter}/ABCD_{graph_num}.gml'
        
        ### Check if the file exists
        if not os.path.exists(gml_path):
            print(f"File {gml_path} not found, skipping...")
            continue
                
        G = nx.read_gml(gml_path)

        ### Run algorithms and store results
        results['MV'].append(MV_Run(G))
        results['MR'].append(MR_Run(G))
        results['Bayan'].append(Bayan_Run(G))
        results['Louvain'].append(Louvain_Run(G))
        if (not (mixing_parameter == 0.5 and graph_num == 2)) and (not (mixing_parameter == 0.7 and graph_num == 25)):
            results['Leiden'].append(Leiden_Run(G))
        results['Paris'].append(Paris_Run(G))
        results['Combo'].append(Combo_Run(G))
        results['Infomap'].append(Infomap_Run(G))
        
    return results

### Run algorithms for different mixing parameters
mixing_parameters = [0.1, 0.3, 0.5, 0.7, 0.9]
for mixing_parameter in mixing_parameters:
    print(f"\n=== Running algorithms for mixing parameter {mixing_parameter} ===")
    results = run_algorithms_on_ABCD(mixing_parameter)
    ### Calculate average AMI scores for each algorithm
    for algorithm in results:
        ### Check if the algorithm in results is not empty array
        if not results[algorithm]:
            print(f"{algorithm} has no results for this mixing parameter")
            continue
        ### Output the average AMI score for the algorithm for this mixing parameter
        average_ami = sum(results[algorithm]) / len(results[algorithm])
        print(f"{algorithm} under xi = {mixing_parameter}: {average_ami:.6f}")
    



=== Running algorithms for mixing parameter 0.1 ===
MV has no results for this mixing parameter
MR has no results for this mixing parameter
Bayan has no results for this mixing parameter
Louvain has no results for this mixing parameter
Leiden has no results for this mixing parameter
Paris has no results for this mixing parameter
Combo has no results for this mixing parameter
Infomap under xi = 0.1: 0.843980

=== Running algorithms for mixing parameter 0.3 ===
MV has no results for this mixing parameter
MR has no results for this mixing parameter
Bayan has no results for this mixing parameter
Louvain has no results for this mixing parameter
Leiden has no results for this mixing parameter
Paris has no results for this mixing parameter
Combo has no results for this mixing parameter
Infomap under xi = 0.3: 0.702958

=== Running algorithms for mixing parameter 0.5 ===
MV has no results for this mixing parameter
MR has no results for this mixing parameter
Bayan has no results for this mixin